In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
import copy
import tensorflow as tf
import scipy as scp
import scipy.misc

import os
import time
import numpy as np
from IPython.display import clear_output
import sys
import csv
import math
from matplotlib import colors
import fcn32_new_wval as fcn32_vgg
import csv

#ckpt_dir = "/mnt/data3/FCN_rel/fine/ckpt_dir"

PRETRAINED_MODEL_PATH= "/mnt/data3/FCN_my_train/ckpt-dir/"

WEIGHT_PATH = "/mnt/data1/imagenet-data/weights/vgg-epoch-78.npy"

TRAINSET_PATH = '/mnt/data1/city/csv_files/cityscapes_train_wmask2.csv'
VALSET_PATH = "/home/sik4hi/city/csv_files/cityscapes_val_wmask.csv"

#data_csv_path = 'fcn32-game-val.csv'


NUM_OF_CLASSESS = 19
BATCH_SIZE = 1
IMAGE_HEIGHT = 1024
IMAGE_WIDTH = 2048
NUM_CHANNELS = 3
N_EPOCHS = 1

os.environ["CUDA_VISIBLE_DEVICES"] = "0"





with tf.device('/gpu:0'):
    sess = tf.Session()
    images = tf.placeholder("float")
    batch_images = tf.expand_dims(images, 0)
    #label_path = tf.placeholder(tf.string)
    #masks_tf = tf.placeholder(tf.float32,[None,2097152])
    train_mode = tf.placeholder(tf.bool)
    
    
    vgg_fcn = fcn32_vgg.FCN32VGG()#('./vgg16.npy')
    
                          
    with tf.name_scope("content_vgg"):
        vgg_fcn.build(batch_images, train=train_mode, num_classes=19, random_init_fc8=False, debug=False)
with tf.device('/cpu:0'):
    saver = tf.train.Saver(max_to_keep=50)
    
with tf.device('/gpu:0'):
    print('Finished building Network.')
    
    init_op = tf.group(tf.initialize_all_variables(),
                       tf.initialize_local_variables())
    sess.run(init_op)
    if PRETRAINED_MODEL_PATH:
        print "using Pretrained model"
        ckpt = tf.train.get_checkpoint_state(PRETRAINED_MODEL_PATH)
        print(ckpt)
        saver.restore(sess,ckpt.model_checkpoint_path)
    print("finished")

In [ ]:
for i in xrange(10)
img1 = scp.misc.imread("/mnt/data3/selected images/images/"+str(i)+".png")
plt.imshow(img1)

In [ ]:
total_train_loss_list,train_loss_list,plot_train_loss,plot_total_train_loss,plot_train_miou= [],[],[],[],[]
val_loss_list,plot_val_loss,plot_val_miou= [],[],[]
#for epoch in xrange(N_EPOCHS):
cmap = colors.ListedColormap([[0.4667,0.0431,0.1254],[0.2745,0.2745,0.2745],[0,0.2353,0.3921]
                              ,[0,0,0.5569],[0.7451,0.6,0.6],[0,0,0.902]
                              ,[0.8627,0.0784,0.2352],[0.6,0.6,0.6],[1,0,0]
                              ,[0.502,0.251,0.502],[0.9569,0.1372,0.9098],[0.2745,0.5098,0.7059]
                              ,[0.5961,0.9843,0.5961],[0.9804,0.6667,0.1176],[0.8627,0.8627,0]
                              ,[0,0.3137,0.3922],[0,0,0.2745],[0.4196,0.5569,0.1372]
                              ,[0.4,0.4,0.6118],[0,0,0]])
norm=colors.NoNorm(0,19)

#while coord.should_stop(): 
for i in xrange(10):
    img1 = scp.misc.imread("/mnt/data3/selected images/images/"+str(i)+".png")
    plt.imshow(img1)
    val_pred_pic = sess.run([vgg_fcn.pred_up],feed_dict={train_mode: False, images: img1})
    plt.figure(1)
    plt.imshow(val_pred_pic[0][0],cmap=cmap,norm=norm)
    image=cmap(norm(val_pred_pic[0][0]))
    plt.imsave("/mnt/data3/selected images/predictions/"+str(i)+".png",image,format='png')

In [ ]:
#print(confusion)
row_sum=np.sum(confusion,axis=0)
col_sum=np.sum(confusion,axis=1)
#print(row_sum)
#print(col_sum)

class_miou=[0 for x in range(19)]
for i in xrange(len(confusion[0])):
    #print(confusion[i][i])
    class_miou[i]=(float(confusion[i][i])/float((row_sum[i]+col_sum[i]-confusion[i][i])))

ofile  = open('mean_iou_random.csv', "w")
writer = csv.writer(ofile)
for i in range(len(class_miou)):
    writer.writerow([i,class_miou[i]])
ofile.close()    
print(class_miou)


